In [26]:
#import des librairies de manipulation de données
from pandas import *
import numpy as np
from scipy import stats, integrate

#import des librairies de visualisation
import seaborn as sns; sns.set(style="ticks", color_codes=True)
import matplotlib as mpl;
import matplotlib.pyplot as plt
%matplotlib inline

In [27]:
#on charge le csv en indiquant que le séparateur est la virgule et que la première ligne contient les en-têtes
df = read_csv('CIMADE_cleaned_v1.csv', sep=',', header=0)

In [28]:
#on affiche un résumé synthétiques des métadonnées du dataframe: nombre de lignes, types de données etc
df.info()

#on affiche 5 lignes choisies au hasard dans le dataframe
df.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234872 entries, 0 to 234871
Data columns (total 7 columns):
departement      234872 non-null int64
prefecture       234872 non-null object
requete          234872 non-null object
date_sondage     234872 non-null object
heure_sondage    234872 non-null object
proposition1     234872 non-null object
proposition2     234872 non-null object
dtypes: int64(1), object(6)
memory usage: 12.5+ MB


,departement,prefecture,requete,date_sondage,heure_sondage,proposition1,proposition2
71056,94,Préfecture de Créteil,Demande de naturalisation,2016-02-12T00:00:00Z,09:25:41,,
117984,13,Sous-préfecture d'Arles,Rendez-vous pour les associations loi 1901,2016-03-15T00:00:00Z,01:09:03,2016-03-15,2016-03-16
153105,92,Sous-préfecture de Boulogne,Échange de permis de conduire étranger,2016-04-07T00:00:00Z,08:30:56,2016-04-12,2016-04-14
31767,29,Sous-préfecture de Brest,Première demande et renouvellement de titre de...,2016-01-08T00:00:00Z,17:07:54,,
194413,30,Préfecture de Nîmes,"Première demande de titre de séjour, sauf admi...",2016-05-03T00:00:00Z,21:11:44,,


In [29]:
#on nettoie les données de date_sondage et on la fusionne avec heure_sondage
df['date_sondage'] = df['date_sondage'].str.replace('T00:00:00Z',' ') + df['heure_sondage']
df = df.drop('heure_sondage',1)

#on affiche les 5 premières lignes du nouveau dataframe
df.head(5)

,departement,prefecture,requete,date_sondage,proposition1,proposition2
0,91,Préfecture d'Évry,Renouvellement de carte de séjour « étudiant »,2015-11-29 20:07:04,,
1,91,Préfecture d'Évry,Renouvellement de la carte de séjour « scienti...,2015-11-29 20:07:06,,
2,91,Préfecture d'Évry,Renouvellement de carte de séjour « conjoint d...,2015-11-29 20:07:07,,
3,91,Préfecture d'Évry,Renouvellement de carte de séjour « mineur pri...,2015-11-29 20:07:09,,
4,91,Préfecture d'Évry,Renouvellement de carte de séjour « mineur pri...,2015-11-29 20:07:10,,


In [30]:
#on utilise to.datetime pour assigner un type adapté aux données temporelle
df['date_sondage'] = to_datetime(df['date_sondage'])
df['proposition1'] = to_datetime(df['proposition1'], errors='coerce')
df['proposition2'] = to_datetime(df['proposition2'], errors='coerce')

#on vérifie les différents types du dataframe
df.dtypes

departement              int64
prefecture              object
requete                 object
date_sondage    datetime64[ns]
proposition1    datetime64[ns]
proposition2    datetime64[ns]
dtype: object

In [31]:
#on assigne date_sondage à l'index
df.index = df['date_sondage']

df.head(5)

,departement,prefecture,requete,date_sondage,proposition1,proposition2
date_sondage,,,,,,
2015-11-29 20:07:04,91,Préfecture d'Évry,Renouvellement de carte de séjour « étudiant »,2015-11-29 20:07:04,NaT,NaT
2015-11-29 20:07:06,91,Préfecture d'Évry,Renouvellement de la carte de séjour « scienti...,2015-11-29 20:07:06,NaT,NaT
2015-11-29 20:07:07,91,Préfecture d'Évry,Renouvellement de carte de séjour « conjoint d...,2015-11-29 20:07:07,NaT,NaT
2015-11-29 20:07:09,91,Préfecture d'Évry,Renouvellement de carte de séjour « mineur pri...,2015-11-29 20:07:09,NaT,NaT
2015-11-29 20:07:10,91,Préfecture d'Évry,Renouvellement de carte de séjour « mineur pri...,2015-11-29 20:07:10,NaT,NaT


In [32]:
df_grouped = df.groupby(TimeGrouper('1M'))

In [33]:
df_grouped.count()

,departement,prefecture,requete,date_sondage,proposition1,proposition2
date_sondage,,,,,,
2015-11-30,292,292,292,292,117,115
2015-12-31,22690,22690,22690,22690,9647,8112
2016-01-31,34617,34617,34617,34617,15446,13083
2016-02-29,39358,39358,39358,39358,16751,14839
2016-03-31,47007,47007,47007,47007,20474,18318
2016-04-30,46092,46092,46092,46092,21092,18699
2016-05-31,44816,44816,44816,44816,22005,19370


In [44]:
df_grouped2 = df.groupby(['prefecture', 'requete']).count()
df_grouped2.unstack()

departement  \
requete                                   Demande de naturalisation   
prefecture                                                            
Préfecture d'Orléans                                            NaN   
Préfecture d'Évry                                               NaN   
Préfecture de Bobigny                                        3842.0   
Préfecture de Chartres                                          NaN   
Préfecture de Créteil                                        3816.0   
Préfecture de Lyon                                              NaN   
Préfecture de Mets                                              NaN   
Préfecture de Montpellier                                       NaN   
Préfecture de Nanterre                                       3610.0   
Préfecture de Nîmes                                             NaN   
Préfecture de Quimper                                           NaN   
Préfecture de Rouen                                             NaN   
Sous-préfecture d'Aix                                           NaN   
Sous-préfecture d'Alès                                          NaN   
Sous-préfecture d'Antony                                        NaN   
Sous-préfecture d'Arles                                         NaN   
Sous-préfecture d'Étampes                                       NaN   
Sous-préfecture de Boulogne                                     NaN   
Sous-préfecture de Brest                                        NaN   
Sous-préfecture de Béziers                                      NaN   
Sous-préfecture de Douai                                        NaN   
Sous-préfecture de Dreux                                        NaN   
Sous-préfecture de Forbach/Boulay-Moselle                       NaN   
Sous-préfecture de L'Haÿ-les-Roses                           3839.0   
Sous-préfecture de Montargis                                    NaN   
Sous-préfecture de Nogent-sur-Marne                          3840.0   
Sous-préfecture de Palaiseau                                    NaN   
Sous-préfecture de Saint-Denis                               3831.0   
Sous-préfecture de Sarcelles                                    NaN   
Sous-préfecture de Thionville                                   NaN   
Sous-préfecture de Vénissieux                                   NaN   
Sous-préfecture du Havre                                        NaN   
Sous-préfecture du Raincy                                    3835.0   

                                                                                                       \
requete                                   Demande d’acquisition de la nationalité à raison du mariage   
prefecture                                                                                              
Préfecture d'Orléans                                                                     NaN            
Préfecture d'Évry                                                                        NaN            
Préfecture de Bobigny                                                                    NaN            
Préfecture de Chartres                                                                   NaN            
Préfecture de Créteil                                                                 3820.0            
Préfecture de Lyon                                                                       NaN            
Préfecture de Mets                                                                       NaN            
Préfecture de Montpellier                                                                NaN            
Préfecture de Nanterre                                                                3622.0            
Préfecture de Nîmes                                                                      NaN            
Préfecture de Quimper                                                                    NaN            
Préfecture de Rouen          

In [37]:
df_grouped2.head

<bound method NDFrame.head of                                                                                               departement  \
prefecture                                requete                                                           
Préfecture d'Orléans                      Première demande de titre de séjour et renouvel...         4030   
Préfecture d'Évry                         Rendez-vous pour les associations loi 1901                 4132   
                                          Renouvellement de carte de séjour « conjoint de...         4128   
                                          Renouvellement de carte de séjour « mineur pris...         4132   
                                          Renouvellement de carte de séjour « mineur pris...         4125   
                                          Renouvellement de carte de séjour « salarié »              4123   
                                          Renouvellement de carte de séjour « étudiant »          